In [106]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_dark"
from plotly.subplots import make_subplots
import dash
import dash_core_components as dcc
import dash_html_components as html
from pathlib import Path
data_dir = Path('D:\PycharmProjects\Covid19\data_dir')

import os
os.listdir(data_dir)
app =dash.Dash()

In [107]:
data = pd.read_csv(data_dir/'covid_19_clean_complete.csv', parse_dates=['Date'],engine='python')
data.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0,0,0
1,NaN,Albania,41.1533,20.1683,2020-01-22,0,0,0
2,NaN,Algeria,28.0339,1.6596,2020-01-22,0,0,0
3,NaN,Andorra,42.5063,1.5218,2020-01-22,0,0,0
4,NaN,Angola,-11.2027,17.8739,2020-01-22,0,0,0


In [108]:
data.rename(columns={'Date': 'date', 
                     'Province/State':'state',
                     'Country/Region':'country',
                     'Confirmed': 'confirmed',
                     'Deaths':'deaths',
                     'Recovered':'recovered'
                    }, inplace=True)


cases = ['confirmed', 'deaths', 'recovered', 'active']

# Active Case = confirmed - deaths - recovered
data['active'] = data['confirmed'] - data['deaths'] - data['recovered']

# replacing Mainland china with just China
data['country'] = data['country'].replace('Mainland China', 'China')

# filling missing values 
data[['state']] = data[['state']].fillna('')
data[cases] = data[cases].fillna(0)
data.rename(columns={'Date':'date'}, inplace=True)

In [109]:
def world_plot(type,color):
    grouped = data.groupby('date')['date', 'confirmed', 'deaths','active' , 'recovered'].sum().reset_index()

    fig = go.Figure()

    if(type == "confirmed + active"):
        fig.add_trace(go.Scatter(x = grouped["date"],y = grouped['confirmed'],
                                 mode = 'lines+markers',line=dict(color="#003EA9"),name="Confimed Cases"))
        
        
        fig.add_trace(go.Scatter(x = grouped["date"],y = grouped['active'],
                                 mode = 'lines+markers',line=dict(color="#44AFFE"),name="Active Cases"))
        
        
        fig.update_layout(title="Worldwide " + type + " Cases",
                     xaxis_title = "Date",
                      yaxis_title = type + " Cases",
                      font = dict(color = "#7f7f7f")
                     )
    else:
        fig.add_trace(go.Scatter(x = grouped["date"],y = grouped[type],
                                 mode = 'lines+markers',line=dict(color=color),name=type + " Cases"))
        fig.update_layout(title="Worldwide " + type + " Cases",
                         xaxis_title = "Date",
                          yaxis_title = type + " Cases",
                          font = dict(color = "#7f7f7f")
                         )
    return fig

In [110]:
data['state'] = data['state'].fillna('')
temp = data[[col for col in data.columns if col != 'state']]

latest = temp[temp['date'] == max(temp['date'])].reset_index()
latest_grouped = latest.groupby('country')['confirmed', 'deaths','active','recovered'].sum().reset_index()

D:\Softwares\Anaconda\envs\covid\lib\site-packages\ipykernel_launcher.py:5: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [111]:
def world_heatmap(type,color,scale):
    avg = latest_grouped[type].sum()/latest_grouped[type].count()
    max = latest_grouped[type].max()
    print(max)
    fig = px.choropleth(latest_grouped, locations="country", 
                        locationmode='country names', color=type, 
                        hover_name="country", range_color=[0,max*scale],
                        color_continuous_midpoint = avg,
                        color_continuous_scale=color, 
                        title="Countries with "+type +" Cases",height=1000)
    return fig
# fig.update(layout_coloraxis_showscale=False)

In [124]:
def world_animation(type,color):
    formated_gdf = data.groupby(['date', 'country'])['confirmed', 'deaths', 'active', 'recovered'].max()
    formated_gdf = formated_gdf.reset_index()
    formated_gdf['date'] = pd.to_datetime(formated_gdf['date'])
    formated_gdf['date'] = formated_gdf['date'].dt.strftime('%m/%d/%Y')
    formated_gdf['size'] = formated_gdf['recovered'].pow(0.3)
    
    fig = px.scatter_geo(formated_gdf, locations="country", locationmode='country names', 
                     color=type, size='size', hover_name="country", 
                     range_color= [0, 100], 
                     projection="natural earth", animation_frame="date", 
                     title='COVID-19: '+type +' Over Time', color_continuous_scale=color)
#     fig.update(layout_coloraxis_showscale=False)
    return fig


In [125]:
color_confirmed =['white','#CCD4DF','#AEBFD5','#96D1D3','#7898C0','#6287B6','#4B75A8','#3A679F',
                           '#2B5697','#0A3B78']
color_active = ['white','#EAD5F0','#DEBBE9','#CF9BDF','#C884DE','#B863D2','#AE4ACD','#9E31C0','#9621BB','#8A07B3']

color_death = ['white','#FEE2DD','#FED0C7','#FEB5A8','#FFA190','#FE9683','#FE8670','#FB745C',
                           '#F3674E','#E95A40']

color_recovered = ['white','#E6F6B7','#E9FCAA','#E5FB9F','#E3FC91','#DBFE6F','#E9FE64','#D2FD50','#D3FF47','#CCFB39',
                   '#BDEA16']
world_confirmed_heatmap = world_heatmap('confirmed',color_confirmed,0.2)
world_active_heatmap = world_heatmap('active',color_active,scale=0.2)
world_deaths_heatmap = world_heatmap('deaths',color_death,scale=0.2)
world_recovered_heatmap = world_heatmap('recovered',color_recovered,0.2)

83836
81946
8215
74181


In [126]:
fig_con_act = world_plot('confirmed + active',"#44AFFE")

fig_death = world_plot('deaths','red')
fig_recovered = world_plot('recovered','green')

D:\Softwares\Anaconda\envs\covid\lib\site-packages\ipykernel_launcher.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [127]:
world_animation_confirm = world_animation('confirmed',color_confirmed)
world_animation_deaths= world_animation('deaths',color_death)
world_animation_active = world_animation('active',color_active)
world_animation_recovered = world_animation('recovered',color_recovered)

D:\Softwares\Anaconda\envs\covid\lib\site-packages\ipykernel_launcher.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [115]:
app.layout = html.Div([
     html.H1("Confirm Cases Heat Map"),
    html.Div(
    dcc.Graph(id = "Confirmed_heatmap",figure = world_confirmed_heatmap)
    ),
    
     html.H1(" Active Cases Heat Map"),
    html.Div(
    dcc.Graph(id = "active_heatmap",figure = world_active_heatmap)
    ),
    
     html.H1("Death Cases Heat Map"),
    html.Div(
    dcc.Graph(id = "death_heatmap",figure = world_deaths_heatmap)
    ),
    
     html.H1("Recovered Cases heat Map"),
    html.Div(
    dcc.Graph(id = "recovered_heatmap",figure = world_recovered_heatmap)
    ),
    
    html.H1("Confirm and Active Cases"),
    html.Div(
    dcc.Graph(id = "Confirmed",figure = fig_con_act)
    ),
    
    html.H1("Worldwide Deaths"),
    html.Div(
        dcc.Graph(id="deaths", figure=fig_death),
    ),
    
    html.H1("Worldwide Recovered"),
        html.Div(
            dcc.Graph(id="recovered", figure=fig_recovered),
        )
])
if __name__ == "__main__":
    app.run_server(debug=True,port=8080,host="10.145.142.110",use_reloader = False)


Running on http://10.145.142.110:8080/
Running on http://10.145.142.110:8080/
Running on http://10.145.142.110:8080/
Running on http://10.145.142.110:8080/
Running on http://10.145.142.110:8080/
Debugger PIN: 173-559-948
Debugger PIN: 173-559-948
Debugger PIN: 173-559-948
Debugger PIN: 173-559-948
Debugger PIN: 173-559-948
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
